In [1]:
# ABOUTME: Background subtraction POC for fast fighter detection in Bloody Roar II
# ABOUTME: Tests speed and reliability of MOG2 background subtraction vs YOLO

import cv2
import numpy as np
from mss import mss
import win32gui
import time
import json
from datetime import datetime
from collections import deque

class BackgroundSubtractionTest:
    def __init__(self, window_title, show_visualization=True):
        self.window_title = window_title
        self.show_visualization = show_visualization
        self.sct = mss()
        
        # Background subtraction setup
        self.bg_subtractor = cv2.createBackgroundSubtractorMOG2(
            history=500,        # Number of frames to build background model
            varThreshold=50,    # Threshold for foreground detection
            detectShadows=True  # Detect shadows
        )
        
        # Fighter detection parameters
        self.min_fighter_area = 800    # Minimum area for a fighter
        self.max_fighter_area = 8000   # Maximum area for a fighter
        self.min_fighter_width = 20    # Minimum width
        self.min_fighter_height = 40   # Minimum height
        
        # Tracking for P1/P2 assignment
        self.previous_positions = []
        self.position_history = deque(maxlen=10)
        
        # Performance tracking
        self.frame_times = []
        self.detection_times = []
        self.total_frames = 0
        
        # Get window handle and dimensions
        self.hwnd = win32gui.FindWindow(None, self.window_title)
        if not self.hwnd:
            raise RuntimeError(f"Window not found: {self.window_title}")
            
        rect = win32gui.GetClientRect(self.hwnd)
        self.left, self.top = win32gui.ClientToScreen(self.hwnd, (0, 0))
        self.width = rect[2]
        self.height = rect[3]
        
        # Health detection (keep from original)
        self.health_params = {
            'p1_x': 505,
            'p2_x': 1421,
            'bar_len': 400,
            'y': 155,
            'lower_bgr': np.array([0, 160, 190], dtype=np.uint8),
            'upper_bgr': np.array([20, 180, 220], dtype=np.uint8),
            'drop_per_px': 0.25
        }
        
        print(f"Background Subtraction Test initialized")
        print(f"Window: {self.width}x{self.height}")
        print(f"Fighter area range: {self.min_fighter_area} - {self.max_fighter_area} pixels")
        
    def detect_health(self):
        """Detect health bars using existing method."""
        # ROIs for health bars
        roi_p1 = {
            'left': self.left + self.health_params['p1_x'],
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        roi_p2 = {
            'left': self.left + (self.health_params['p2_x'] - self.health_params['bar_len']),
            'top': self.top + self.health_params['y'],
            'width': self.health_params['bar_len'],
            'height': 1,
        }
        
        # Capture and process health bars
        try:
            raw_p1 = self.sct.grab(roi_p1)
            strip_p1 = np.array(raw_p1)[:, :, :3]
            b1, g1, r1 = strip_p1[0].T
            
            raw_p2 = self.sct.grab(roi_p2)
            strip_p2 = np.array(raw_p2)[:, :, :3]
            b2, g2, r2 = strip_p2[0].T
            
            # Process P1 health
            mask_p1 = (
                (r1 >= self.health_params['lower_bgr'][2]) & 
                (r1 <= self.health_params['upper_bgr'][2]) &
                (g1 >= self.health_params['lower_bgr'][1]) & 
                (g1 <= self.health_params['upper_bgr'][1]) &
                (b1 >= self.health_params['lower_bgr'][0]) & 
                (b1 <= self.health_params['upper_bgr'][0])
            )
            non_yellow_p1 = np.nonzero(~mask_p1)[0]
            last_idx_p1 = non_yellow_p1.max() if non_yellow_p1.size else -1
            drop_pixels_p1 = max(0, last_idx_p1 + 1)
            life_pct_p1 = 100.0 - (drop_pixels_p1 * self.health_params['drop_per_px'])
            life_pct_p1 = np.clip(life_pct_p1, 0.0, 100.0)
            
            # Process P2 health
            mask_p2 = (
                (r2 >= self.health_params['lower_bgr'][2]) & 
                (r2 <= self.health_params['upper_bgr'][2]) &
                (g2 >= self.health_params['lower_bgr'][1]) & 
                (g2 <= self.health_params['upper_bgr'][1]) &
                (b2 >= self.health_params['lower_bgr'][0]) & 
                (b2 <= self.health_params['upper_bgr'][0])
            )
            non_yellow_p2 = np.nonzero(~mask_p2)[0]
            last_idx_p2 = non_yellow_p2.min() if non_yellow_p2.size else self.health_params['bar_len']
            drop_pixels_p2 = self.health_params['bar_len'] - last_idx_p2
            life_pct_p2 = 100.0 - (drop_pixels_p2 * self.health_params['drop_per_px'])
            life_pct_p2 = np.clip(life_pct_p2, 0.0, 100.0)
            
            return life_pct_p1, life_pct_p2
            
        except Exception as e:
            print(f"Health detection error: {e}")
            return 0.0, 0.0
    
    def detect_fighters_background_subtraction(self, frame):
        """Detect fighters using background subtraction."""
        detection_start = time.perf_counter()
        
        # Apply background subtraction
        fg_mask = self.bg_subtractor.apply(frame)
        
        # Clean up the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter contours by size and shape
        fighter_candidates = []
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < self.min_fighter_area or area > self.max_fighter_area:
                continue
                
            # Get bounding rectangle
            x, y, w, h = cv2.boundingRect(contour)
            
            # Filter by dimensions
            if w < self.min_fighter_width or h < self.min_fighter_height:
                continue
                
            # Calculate aspect ratio (fighters are typically taller than wide)
            aspect_ratio = h / w
            if aspect_ratio < 0.8:  # Too wide to be a fighter
                continue
            
            # Calculate center point
            center_x = x + w // 2
            center_y = y + h // 2
            
            fighter_candidates.append({
                'center': (center_x, center_y),
                'area': area,
                'bbox': (x, y, w, h),
                'contour': contour
            })
        
        # Sort by area (larger objects more likely to be fighters)
        fighter_candidates.sort(key=lambda x: x['area'], reverse=True)
        
        # Assign to P1 and P2 using tracking
        p1_pos, p2_pos = self.assign_players(fighter_candidates)
        
        detection_time = (time.perf_counter() - detection_start) * 1000
        self.detection_times.append(detection_time)
        
        return p1_pos, p2_pos, fg_mask, fighter_candidates
    
    def assign_players(self, candidates):
        """Assign detected fighters to P1 and P2 using position tracking."""
        if not candidates:
            return None, None
            
        if len(candidates) == 1:
            # Only one fighter detected, assign based on previous position
            pos = candidates[0]['center']
            if self.previous_positions:
                # Find which previous position is closer
                prev_p1, prev_p2 = self.previous_positions[-1]
                if prev_p1 and prev_p2:
                    dist_to_p1 = np.linalg.norm(np.array(pos) - np.array(prev_p1))
                    dist_to_p2 = np.linalg.norm(np.array(pos) - np.array(prev_p2))
                    if dist_to_p1 < dist_to_p2:
                        return pos, None
                    else:
                        return None, pos
                elif prev_p1:
                    return pos, None
                elif prev_p2:
                    return None, pos
            
            # No previous position, assign based on screen position
            if pos[0] < self.width // 2:
                return pos, None
            else:
                return None, pos
        
        # Multiple candidates - assign based on x-position and tracking
        if len(candidates) >= 2:
            # Sort by x-position
            candidates.sort(key=lambda x: x['center'][0])
            
            # If we have previous positions, use tracking
            if self.previous_positions:
                prev_p1, prev_p2 = self.previous_positions[-1]
                if prev_p1 and prev_p2:
                    # Match based on minimum distance
                    best_assignment = self.find_best_assignment(candidates, prev_p1, prev_p2)
                    if best_assignment:
                        return best_assignment
            
            # Default: leftmost is P1, rightmost is P2
            return candidates[0]['center'], candidates[1]['center']
        
        return None, None
    
    def find_best_assignment(self, candidates, prev_p1, prev_p2):
        """Find the best assignment of candidates to P1 and P2."""
        if len(candidates) < 2:
            return None
            
        # Try all possible assignments and pick the one with minimum total distance
        best_assignment = None
        min_total_distance = float('inf')
        
        for i, c1 in enumerate(candidates):
            for j, c2 in enumerate(candidates):
                if i == j:
                    continue
                    
                # Calculate distance if c1->P1 and c2->P2
                dist1 = np.linalg.norm(np.array(c1['center']) - np.array(prev_p1))
                dist2 = np.linalg.norm(np.array(c2['center']) - np.array(prev_p2))
                total_dist = dist1 + dist2
                
                if total_dist < min_total_distance:
                    min_total_distance = total_dist
                    best_assignment = (c1['center'], c2['center'])
        
        return best_assignment
    
    def get_game_state(self):
        """Get current game state using background subtraction."""
        frame_start = time.perf_counter()
        
        # Capture screen
        monitor = {
            'left': self.left,
            'top': self.top,
            'width': self.width,
            'height': self.height
        }
        
        screenshot = np.array(self.sct.grab(monitor))
        frame = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        
        # Detect fighters
        p1_pos, p2_pos, fg_mask, candidates = self.detect_fighters_background_subtraction(frame)
        
        # Update position history
        self.previous_positions.append((p1_pos, p2_pos))
        if len(self.previous_positions) > 10:
            self.previous_positions.pop(0)
        
        # Get health
        p1_health, p2_health = self.detect_health()
        
        # Calculate distance
        distance = 0
        if p1_pos and p2_pos:
            distance = abs(p1_pos[0] - p2_pos[0])
        
        frame_time = (time.perf_counter() - frame_start) * 1000
        self.frame_times.append(frame_time)
        self.total_frames += 1
        
        return {
            'p1_pos': p1_pos,
            'p2_pos': p2_pos,
            'p1_health': p1_health,
            'p2_health': p2_health,
            'distance': distance,
            'frame': frame,
            'fg_mask': fg_mask,
            'candidates': candidates,
            'frame_time_ms': frame_time,
            'detection_time_ms': self.detection_times[-1] if self.detection_times else 0
        }
    
    def run_speed_test(self, num_frames=100):
        """Run speed test for specified number of frames."""
        print(f"\nRunning speed test for {num_frames} frames...")
        print("Let the background subtractor learn for a few seconds...")
        
        # Learning phase
        for i in range(50):
            state = self.get_game_state()
            if i % 10 == 0:
                print(f"Learning frame {i}/50")
        
        # Reset timing arrays
        self.frame_times = []
        self.detection_times = []
        
        # Test phase
        print("Starting speed test...")
        for i in range(num_frames):
            state = self.get_game_state()
            
            if i % 10 == 0:
                current_avg = np.mean(self.frame_times[-10:]) if self.frame_times else 0
                print(f"Frame {i}/{num_frames} - Avg time: {current_avg:.2f}ms")
        
        # Calculate statistics
        avg_frame_time = np.mean(self.frame_times)
        avg_detection_time = np.mean(self.detection_times)
        std_frame_time = np.std(self.frame_times)
        fps = 1000 / avg_frame_time if avg_frame_time > 0 else 0
        
        print(f"\n=== SPEED TEST RESULTS ===")
        print(f"Total frames processed: {num_frames}")
        print(f"Average frame time: {avg_frame_time:.2f}ms (±{std_frame_time:.2f}ms)")
        print(f"Average detection time: {avg_detection_time:.2f}ms")
        print(f"Average FPS: {fps:.1f}")
        print(f"Min frame time: {min(self.frame_times):.2f}ms")
        print(f"Max frame time: {max(self.frame_times):.2f}ms")
        print(f"95th percentile: {np.percentile(self.frame_times, 95):.2f}ms")
        
        return {
            'avg_frame_time': avg_frame_time,
            'avg_detection_time': avg_detection_time,
            'fps': fps,
            'std_frame_time': std_frame_time,
            'min_time': min(self.frame_times),
            'max_time': max(self.frame_times)
        }
    
    def run_visualization(self):
        """Run with real-time visualization."""
        if not self.show_visualization:
            return
            
        cv2.namedWindow('Background Subtraction Test', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Background Subtraction Test', self.width // 2, self.height // 2)
        
        cv2.namedWindow('Foreground Mask', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Foreground Mask', self.width // 3, self.height // 3)
        
        print("Visualization started. Press 'q' to quit, 's' for speed test")
        
        try:
            while True:
                state = self.get_game_state()
                
                # Draw on main frame
                display = state['frame'].copy()
                
                # Draw detected fighters
                if state['p1_pos']:
                    cv2.circle(display, state['p1_pos'], 10, (0, 255, 0), -1)
                    cv2.putText(display, f"P1: {state['p1_health']:.1f}%", 
                               (state['p1_pos'][0]-50, state['p1_pos'][1]-30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                if state['p2_pos']:
                    cv2.circle(display, state['p2_pos'], 10, (0, 0, 255), -1)
                    cv2.putText(display, f"P2: {state['p2_health']:.1f}%", 
                               (state['p2_pos'][0]-50, state['p2_pos'][1]-30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                # Draw all candidates
                for candidate in state['candidates']:
                    x, y, w, h = candidate['bbox']
                    cv2.rectangle(display, (x, y), (x+w, y+h), (255, 255, 0), 2)
                    cv2.putText(display, f"{candidate['area']}", 
                               (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
                
                # Draw performance info
                if state['frame_time_ms']:
                    cv2.putText(display, f"Frame: {state['frame_time_ms']:.1f}ms", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                    cv2.putText(display, f"Detection: {state['detection_time_ms']:.1f}ms", 
                               (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                if state['distance']:
                    cv2.putText(display, f"Distance: {state['distance']}", 
                               (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                # Show frames
                cv2.imshow('Background Subtraction Test', display)
                cv2.imshow('Foreground Mask', state['fg_mask'])
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('s'):
                    self.run_speed_test(50)
                    
        except KeyboardInterrupt:
            print("\nStopped by user")
        finally:
            cv2.destroyAllWindows()


if __name__ == "__main__":
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    
    try:
        # Test with visualization
        print("=== Background Subtraction Speed Test ===")
        print("This will test background subtraction for fighter detection")
        print("1. Let background model learn (50 frames)")
        print("2. Run speed test (100 frames)")
        print("3. Show visualization")
        
        test = BackgroundSubtractionTest(WINDOW_TITLE, show_visualization=True)
        
        # Run automated speed test first
        results = test.run_speed_test(100)
        
        # Then run visualization
        print("\nStarting visualization...")
        test.run_visualization()
        
    except RuntimeError as e:
        print(f"Error: {e}")
        print("Make sure the game window is open and visible")

=== Background Subtraction Speed Test ===
This will test background subtraction for fighter detection
1. Let background model learn (50 frames)
2. Run speed test (100 frames)
3. Show visualization
Background Subtraction Test initialized
Window: 1919x955
Fighter area range: 800 - 8000 pixels

Running speed test for 100 frames...
Let the background subtractor learn for a few seconds...
Learning frame 0/50
Learning frame 10/50
Learning frame 20/50
Learning frame 30/50
Learning frame 40/50
Starting speed test...
Frame 0/100 - Avg time: 183.60ms
Frame 10/100 - Avg time: 154.13ms
Frame 20/100 - Avg time: 162.42ms
Frame 30/100 - Avg time: 137.15ms
Frame 40/100 - Avg time: 144.01ms
Frame 50/100 - Avg time: 132.40ms
Frame 60/100 - Avg time: 159.36ms
Frame 70/100 - Avg time: 140.38ms
Frame 80/100 - Avg time: 109.67ms
Frame 90/100 - Avg time: 104.94ms

=== SPEED TEST RESULTS ===
Total frames processed: 100
Average frame time: 137.44ms (±26.88ms)
Average detection time: 30.29ms
Average FPS: 7.3
Mi

In [ ]:
# ABOUTME: Game controller for sending actions to BizHawk via file communication
# ABOUTME: Python-side interface for controlling fighters in Bloody Roar II

import time
import os
import json
from datetime import datetime
from enum import IntEnum
from typing import Optional, Dict, Any

class BloodyRoarActions(IntEnum):
    """Action space for Bloody Roar II fighting game."""
    # Basic movements
    IDLE = 0
    LEFT = 1
    RIGHT = 2
    UP = 3          # Jump
    DOWN = 4        # Crouch
    
    # Basic attacks
    PUNCH = 5       # Light attack
    KICK = 6        # Heavy attack
    TRANSFORM = 7   # Beast transformation
    
    # Directional attacks (combos)
    LEFT_PUNCH = 8      # Left + Punch
    RIGHT_PUNCH = 9     # Right + Punch
    LEFT_KICK = 10      # Left + Kick
    RIGHT_KICK = 11     # Right + Kick
    DOWN_PUNCH = 12     # Down + Punch (low attack)
    DOWN_KICK = 13      # Down + Kick (low attack)
    UP_PUNCH = 14       # Up + Punch (jumping attack)
    UP_KICK = 15        # Up + Kick (jumping attack)
    
    # Special combinations
    LEFT_DOWN = 16      # Left + Down (crouch left)
    RIGHT_DOWN = 17     # Right + Down (crouch right)
    PUNCH_KICK = 18     # Punch + Kick (special move)

class GameController:
    def __init__(self, actions_file="actions.txt", latency_log="latency.txt"):
        self.actions_file = actions_file
        self.latency_log = latency_log
        
        # Action mapping for BizHawk
        self.action_map = {
            BloodyRoarActions.IDLE: {"keys": [], "description": "No input"},
            BloodyRoarActions.LEFT: {"keys": ["Left"], "description": "Move left"},
            BloodyRoarActions.RIGHT: {"keys": ["Right"], "description": "Move right"},
            BloodyRoarActions.UP: {"keys": ["Up"], "description": "Jump"},
            BloodyRoarActions.DOWN: {"keys": ["Down"], "description": "Crouch"},
            BloodyRoarActions.PUNCH: {"keys": ["X"], "description": "Punch"},
            BloodyRoarActions.KICK: {"keys": ["Circle"], "description": "Kick"},
            BloodyRoarActions.TRANSFORM: {"keys": ["Square"], "description": "Transform"},
            BloodyRoarActions.LEFT_PUNCH: {"keys": ["Left", "X"], "description": "Left + Punch"},
            BloodyRoarActions.RIGHT_PUNCH: {"keys": ["Right", "X"], "description": "Right + Punch"},
            BloodyRoarActions.LEFT_KICK: {"keys": ["Left", "Circle"], "description": "Left + Kick"},
            BloodyRoarActions.RIGHT_KICK: {"keys": ["Right", "Circle"], "description": "Right + Kick"},
            BloodyRoarActions.DOWN_PUNCH: {"keys": ["Down", "X"], "description": "Down + Punch"},
            BloodyRoarActions.DOWN_KICK: {"keys": ["Down", "Circle"], "description": "Down + Kick"},
            BloodyRoarActions.UP_PUNCH: {"keys": ["Up", "X"], "description": "Up + Punch"},
            BloodyRoarActions.UP_KICK: {"keys": ["Up", "Circle"], "description": "Up + Kick"},
            BloodyRoarActions.LEFT_DOWN: {"keys": ["Left", "Down"], "description": "Left + Down"},
            BloodyRoarActions.RIGHT_DOWN: {"keys": ["Right", "Down"], "description": "Right + Down"},
            BloodyRoarActions.PUNCH_KICK: {"keys": ["X", "Circle"], "description": "Punch + Kick"}
        }
        
        # Performance tracking
        self.action_history = []
        self.latency_measurements = []
        
        # Clear existing action file
        self.clear_actions()
        
        print(f"Game Controller initialized")
        print(f"Action file: {self.actions_file}")
        print(f"Latency log: {self.latency_log}")
        print(f"Available actions: {len(self.action_map)}")
        
    def clear_actions(self):
        """Clear the actions file."""
        try:
            with open(self.actions_file, 'w') as f:
                f.write("")  # Clear file
        except Exception as e:
            print(f"Warning: Could not clear actions file: {e}")
    
    def send_action(self, action: BloodyRoarActions, player: int = 1, duration_frames: int = 1) -> bool:
        """
        Send an action to the game.
        
        Args:
            action: Action to perform (from BloodyRoarActions enum)
            player: Player number (1 or 2)
            duration_frames: How many frames to hold the action
            
        Returns:
            True if action was sent successfully
        """
        if action not in self.action_map:
            print(f"Invalid action: {action}")
            return False
        
        if player not in [1, 2]:
            print(f"Invalid player: {player}")
            return False
        
        timestamp = time.time()
        
        # Create action command
        action_data = {
            "timestamp": timestamp,
            "player": player,
            "action": int(action),
            "keys": self.action_map[action]["keys"],
            "duration": duration_frames,
            "description": self.action_map[action]["description"]
        }
        
        try:
            # Write action to file (Lua will read this)
            with open(self.actions_file, 'w') as f:
                json.dump(action_data, f)
            
            # Track action history
            self.action_history.append(action_data)
            
            # Keep only last 100 actions
            if len(self.action_history) > 100:
                self.action_history.pop(0)
            
            return True
            
        except Exception as e:
            print(f"Error sending action: {e}")
            return False
    
    def send_action_sequence(self, actions: list, player: int = 1, frame_delay: int = 1) -> bool:
        """
        Send a sequence of actions with delays.
        
        Args:
            actions: List of (action, duration) tuples
            player: Player number
            frame_delay: Frames to wait between actions
            
        Returns:
            True if all actions were sent successfully
        """
        for action, duration in actions:
            if not self.send_action(action, player, duration):
                return False
            
            # Wait for action to complete + delay
            time.sleep((duration + frame_delay) * 0.0167)  # Assume 60 FPS
        
        return True
    
    def get_action_description(self, action: BloodyRoarActions) -> str:
        """Get human-readable description of an action."""
        if action in self.action_map:
            return self.action_map[action]["description"]
        return "Unknown action"
    
    def get_all_actions(self) -> Dict[int, str]:
        """Get all available actions with descriptions."""
        return {int(action): data["description"] for action, data in self.action_map.items()}
    
    def log_latency(self, sent_time: float, executed_time: float):
        """Log latency measurement from Lua script."""
        latency_ms = (executed_time - sent_time) * 1000
        self.latency_measurements.append(latency_ms)
        
        # Keep only last 100 measurements
        if len(self.latency_measurements) > 100:
            self.latency_measurements.pop(0)
        
        # Log to file
        try:
            with open(self.latency_log, 'a') as f:
                f.write(f"{datetime.now().isoformat()},{latency_ms:.2f}\n")
        except Exception as e:
            print(f"Warning: Could not log latency: {e}")
    
    def get_latency_stats(self) -> Dict[str, float]:
        """Get latency statistics."""
        if not self.latency_measurements:
            return {"avg": 0, "min": 0, "max": 0, "count": 0}
        
        return {
            "avg": sum(self.latency_measurements) / len(self.latency_measurements),
            "min": min(self.latency_measurements),
            "max": max(self.latency_measurements),
            "count": len(self.latency_measurements)
        }
    
    def test_action_sending(self, num_actions: int = 10):
        """Test action sending with timing measurements."""
        print(f"\nTesting action sending ({num_actions} actions)...")
        
        actions_to_test = [
            BloodyRoarActions.LEFT,
            BloodyRoarActions.RIGHT,
            BloodyRoarActions.PUNCH,
            BloodyRoarActions.KICK,
            BloodyRoarActions.UP
        ]
        
        send_times = []
        
        for i in range(num_actions):
            action = actions_to_test[i % len(actions_to_test)]
            
            start_time = time.time()
            success = self.send_action(action, player=1)
            end_time = time.time()
            
            send_time = (end_time - start_time) * 1000
            send_times.append(send_time)
            
            print(f"Action {i+1}: {self.get_action_description(action)} - "
                  f"Send time: {send_time:.2f}ms - {'✓' if success else '✗'}")
            
            time.sleep(0.1)  # Small delay between actions
        
        # Statistics
        avg_send_time = sum(send_times) / len(send_times)
        print(f"\nSend time statistics:")
        print(f"Average: {avg_send_time:.2f}ms")
        print(f"Min: {min(send_times):.2f}ms")
        print(f"Max: {max(send_times):.2f}ms")
        
        return avg_send_time
    
    def test_random_agent(self, duration_seconds: int = 30):
        """Test with a random agent for demonstration."""
        import random
        
        print(f"\nRunning random agent for {duration_seconds} seconds...")
        print("This will send random actions to test the system")
        print("Make sure BizHawk is running with the Lua script loaded")
        
        start_time = time.time()
        action_count = 0
        
        while time.time() - start_time < duration_seconds:
            # Random action
            action = random.choice(list(BloodyRoarActions))
            
            # Send action
            success = self.send_action(action, player=1)
            if success:
                action_count += 1
                print(f"Action {action_count}: {self.get_action_description(action)}")
            
            # Wait a bit
            time.sleep(0.5)  # 2 actions per second
        
        print(f"\nRandom agent test completed:")
        print(f"Total actions sent: {action_count}")
        print(f"Actions per second: {action_count / duration_seconds:.1f}")
    
    def cleanup(self):
        """Clean up resources."""
        self.clear_actions()
        print("Game controller cleaned up")


class RLGameController(GameController):
    """Extended controller specifically for RL training."""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.last_action = BloodyRoarActions.IDLE
        self.action_count = 0
        
    def step(self, action: int, player: int = 1) -> bool:
        """
        RL-style step function.
        
        Args:
            action: Action ID (0-18)
            player: Player number
            
        Returns:
            True if action was executed successfully
        """
        try:
            rl_action = BloodyRoarActions(action)
            success = self.send_action(rl_action, player)
            
            if success:
                self.last_action = rl_action
                self.action_count += 1
            
            return success
            
        except ValueError:
            print(f"Invalid action ID: {action}")
            return False
    
    def get_action_space_size(self) -> int:
        """Get the size of the action space for RL algorithms."""
        return len(BloodyRoarActions)
    
    def get_last_action(self) -> BloodyRoarActions:
        """Get the last action that was sent."""
        return self.last_action
    
    def reset(self):
        """Reset the controller state."""
        self.last_action = BloodyRoarActions.IDLE
        self.action_count = 0
        self.clear_actions()


if __name__ == "__main__":
    print("=== Game Controller Test ===")
    
    # Create controller
    controller = GameController()
    
    # Show available actions
    print("\nAvailable actions:")
    for action_id, description in controller.get_all_actions().items():
        print(f"  {action_id}: {description}")
    
    # Test action sending
    controller.test_action_sending(5)
    
    # Ask user for interactive test
    print(f"\nController ready!")
    print(f"1. Load 'bizhawk_controller.lua' in BizHawk")
    print(f"2. Start the game")
    print(f"3. Run random agent test or send manual actions")
    
    choice = input("\nRun random agent test? (y/n): ").strip().lower()
    if choice == 'y':
        controller.test_random_agent(10)
    
    controller.cleanup()